In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import os
import numpy as np
import warnings

In [3]:
tradingPairs = {'JBF':'3443','QWF':'2388','HCF':'2498','DBF':'2610','EHF':'1319','IPF':'3035','IIF':'3006','QXF':'2615','PEF':'5425','NAF':'3105'}

In [4]:
stockPath = "/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/stocks"
stockFiles = os.listdir(stockPath)
futuresPath = "/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/futuresOB"
futuresFiles = os.listdir(futuresPath)
stockFiles.remove('.DS_Store')
futuresFiles.remove('.DS_Store')

In [23]:
stockData = {}
futuresData = {}
for file in stockFiles:
  stockData[file] = os.listdir(stockPath + '/' + file)
for file in futuresFiles:
  futuresData[file] = os.listdir(futuresPath + '/' + file)

In [25]:
stockPriceOneMinute = {}
futuresPriceOneMinute = {}

In [ ]:
warnings.filterwarnings("ignore")
for stock in stockData.keys():
  stockPriceOneMinute[stock] = pd.DataFrame([], columns = ['date','time','midQuote'])
  for date in stockData[stock]:
    stockTick = pd.read_csv(stockPath + '/' + stock + '/' + date)
    #Data processing
    stockTick = stockTick[['date','time','SP1','BP1']].dropna()
    stockTick = stockTick.loc[stockTick['SP1'] * stockTick['BP1'] != 0]
    stockTick['midQuote'] = np.log(stockTick[['SP1','BP1']].mean(axis=1))
    stockTick['time'] = (stockTick['time'] - stockTick['time'] % 100000) / 100000
    stockPrice = stockTick[['date','time','midQuote']]
    means = stockPrice.groupby(['date','time'])['midQuote'].mean()
    price_ = pd.DataFrame({'midQuote': means})
    price_ = price_.reset_index()
    print(price_)
    frames = [stockPriceOneMinute[stock], price_]
    stockPriceOneMinute[stock] = pd.concat(frames)

In [33]:
for name in stockData.keys():
  with open('/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/stockData/' + name + '.csv', 'w', encoding = 'utf-8-sig') as f:
    stockPriceOneMinute[name].to_csv(f)

In [28]:
for futures in futuresData.keys():
  futuresPriceOneMinute[futures] = pd.DataFrame([], columns = ['date','time','midQuote'])
  for date in futuresData[futures]:
    futuresTick = pd.read_csv(futuresPath + '/' + futures + '/' + date)
    #Data processing
    futuresTick = futuresTick[['date','time','askPrice1','bidPrice1']].dropna()
    futuresTick = futuresTick.loc[futuresTick['askPrice1'] * futuresTick['bidPrice1'] != 0]
    futuresTick['midQuote'] = np.log(futuresTick[['askPrice1','bidPrice1']].mean(axis=1))
    futuresTick['time'] = (futuresTick['time'] - futuresTick['time'] % 100000) / 100000
    futuresPrice = futuresTick[['date','time','midQuote']]
    means = futuresPrice.groupby(['date','time'])['midQuote'].mean()
    price_ = pd.DataFrame({'midQuote': means})
    price_ = price_.reset_index()
    frames = [futuresPriceOneMinute[futures], price_]
    futuresPriceOneMinute[futures] = pd.concat(frames)

In [32]:
for name in futuresData.keys():
  with open('/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/futuresData/' + name + '.csv', 'w', encoding = 'utf-8-sig') as f:
    futuresPriceOneMinute[name].to_csv(f)

In [ ]:
spreads = {}
for futures in futuresData.keys():
  merged_df = pd.merge(stockPriceOneMinute[tradingPairs[futures]], futuresPriceOneMinute[futures], on=['date','time'], suffixes=('_1', '_2'))
  merged_df['spread'] = merged_df['midQuote_1'] - merged_df['midQuote_2']
  print(merged_df)
  spreads[futures] = merged_df[['date','time','spread']].dropna()

In [45]:
for name in spreads.keys():
  with open('/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/spreadData_new/' + name + '_spread.csv', 'w', encoding = 'utf-8-sig') as f:
    spreads[name].to_csv(f)

In [ ]:
spreads['JBF']

In [ ]:
stockPriceOneMinute['2615']

In [44]:
futuresPriceOneMinute['QXF']

,date,time,midQuote
0,2022-04-27,845.0,9.547991
1,2022-04-27,846.0,9.553930
2,2022-04-27,847.0,9.549608
3,2022-04-27,848.0,9.546813
4,2022-04-27,849.0,9.546813
...,...,...,...
5062,2023-06-30,1340.0,8.690306
5063,2023-06-30,1341.0,8.690306
5064,2023-06-30,1342.0,8.690306
5065,2023-06-30,1343.0,8.690306
